In [15]:
import pandas
from imblearn.over_sampling import SMOTE
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


In [16]:
# data loading
data = scipy.io.loadmat('x.mat')  
columns = np.asarray([['Electrode %d - %d/2 Hz'%(i+1, j+1)] for i in range(data['x'].shape[1]) for j in range(data['x'].shape[2])])
data['x'].shape
labels = pd.read_csv("table_withlabels.csv")
foof = pd.read_csv("foof2features.csv")
beh = pd.read_csv("behaviorals.csv")
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df.columns = columns
df['IDs'] = foof['C1']
df = pd.merge(df, labels[['label', 'IDs']], on='IDs', how='inner')
print(df['label'].value_counts())


Other Neurodevelopmental Disorders    492
ADHD-Inattentive Type                 388
ADHD-Combined Type                    376
Anxiety Disorders                     241
No Diagnosis Given                    203
Depressive Disorders                   85
Name: label, dtype: int64


In [17]:
dataset = df.values
print(dataset.shape)
dataset

(1785, 8297)


array([[7.707834370762612e-12, 1.5122858521010585e-10,
        2.475735394469325e-09, ..., 1.269651050961329e-12,
        'NDARAA075AMK', 'No Diagnosis Given'],
       [0.03798713306980961, 0.16394328412236603, 0.289759115499709, ...,
        8.780451921097665e-10, 'NDARAA112DMH', 'ADHD-Combined Type'],
       [2.1043101829931743e-06, 4.299646894634357e-05,
        0.0005644195457481516, ..., 3.597966369284222e-11,
        'NDARAA117NEJ', 'ADHD-Combined Type'],
       ...,
       [0.0054727674101446855, 0.008816325192535168, 0.01629355679063138,
        ..., 0.0, 'NDARZN610GTY', 'Other Neurodevelopmental Disorders'],
       [0.002658455271881799, 0.005749805544624453, 0.011805375381391148,
        ..., 9.481114398024459e-07, 'NDARZN677EYE',
        'ADHD-Inattentive Type'],
       [0.00048563852963789955, 0.0030489761928298442,
        0.015468356525379479, ..., 0.0, 'NDARZN899JCM',
        'ADHD-Inattentive Type']], dtype=object)

In [18]:
X = dataset[:,0:8295].astype(int) #bez 8295
Y = dataset[:,8296]
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)



In [19]:
# scaling the data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)
X = np.clip(X, -1, 1)


In [20]:
oversample = SMOTE() #oversample to aaccount for the data imbalance
X,dummy_y = oversample.fit_resample(X, dummy_y)

In [21]:
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=8295, activation='relu'))
	model.add(Dense(6, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [22]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=20, batch_size=5, verbose=1)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X , dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

2021-11-13 21:11:48.041702: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-13 21:11:48.050519: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-13 21:11:48.050802: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:06.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-11-13 21:11:48.050970: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-11-13 21:11:48.052024: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-13 21:11:48.053206: I tensorflow/stream_executor/platform/default/d

Epoch 1/20
   5/2656 [..............................] - ETA: 2:05 - loss: 1.6458 - accuracy: 0.6000

2021-11-13 21:11:48.719309: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


2656/2656 [==============================] - 1s 440us/step - loss: 1.8074 - accuracy: 0.1785
Epoch 2/20
2656/2656 [==============================] - 1s 351us/step - loss: 1.7588 - accuracy: 0.1830
Epoch 3/20
2656/2656 [==============================] - 1s 349us/step - loss: 1.7195 - accuracy: 0.2282
Epoch 4/20
2656/2656 [==============================] - 1s 347us/step - loss: 1.6684 - accuracy: 0.2462
Epoch 5/20
2656/2656 [==============================] - 1s 354us/step - loss: 1.6169 - accuracy: 0.2797
Epoch 6/20
2656/2656 [==============================] - 1s 348us/step - loss: 1.5776 - accuracy: 0.3042
Epoch 7/20
2656/2656 [==============================] - 1s 351us/step - loss: 1.5520 - accuracy: 0.2978
Epoch 8/20
2656/2656 [==============================] - 1s 347us/step - loss: 1.5106 - accuracy: 0.3200
Epoch 9/20
2656/2656 [==============================] - 1s 337us/step - loss: 1.4784 - accuracy: 0.3641
Epoch 10/20
2656/2656 [==============================] - 1s 341us/step - lo